# Déclaration

In [1]:
from importlib import reload
import numpy as np
import time
import datetime
import pyautogui
import pandas as pd
import re
import keyboard
import os
import fct
reload(fct)

wave=pd.read_csv('wave.csv')

class Perso():
    """_summary_
    num_perso numéro du perso à séléctionner
    raid faire attention au horaires des maps
    
    """
    def __init__(self,num_perso=2,list_map=0,can_run_T11=False,blue_map=True,raid=False):
        self.num_perso=num_perso
        self.test_in_game()
        self.blue_map=blue_map
        self.actif_raid=raid
        self.name_map=""
        self.map=fct.map.Map(list_map,can_run_T11)

    def test_in_game(self):
        path1=f'screen/launch/test_in_game.png'
        if pyautogui.locateOnScreen(path1, grayscale=False, confidence=0.95) is None :
            if pyautogui.locateOnScreen(f'screen/launch/in_tower.png',grayscale=True,confidence=0.9) is None :
                if fct.launch_game(self.num_perso) is False:
                        fct.close_game("test_in_game")
    
    def is_raid_time(self,creneau1=[15.45,18.15],creneau2=0):
        if self.actif_raid:
            now = datetime.datetime.now()
            if now.hour>= int(creneau1[0]) and now.hour<=creneau1[1]:
                if now.hour==int(creneau1[1]) and now.minute>=creneau1[1]%1*100:
                    return False
                if now.hour>int(creneau1[0]):
                    return True
                if now.hour==int(creneau1[0]) and now.minute>=creneau1[0]%1*100:
                    return True
            if creneau2:      
                if now.hour>= int(creneau2[0]) and now.hour<=creneau2[1]:
                    if now.hour==int(creneau2[1]) and now.minute>=creneau2[1]%1*100:
                        return False
                    if now.hour>int(creneau2[0]):
                        return True
                    if now.hour==int(creneau2[0]) and now.minute>=creneau2[0]%1*100:
                        return True
        return False        
    
    def launch_map(self):
        name_map=self.map.launch_map()             
        if not name_map or not fct.go_map(wave): 
            return False
        self.name_map=name_map
        return True
    
    def find_map_on_start(self,name_map):
        for i in range(10):
            if fct.in_map():break
            time.sleep(1)
        if self.map.can_run_T11: return name_map+'_1'
        fct.click_icone('screen/launch/tab_map.png',1,confidence=0.85)
        pyautogui.moveTo(700,700)
        time.sleep(0.2)
        for i in range(10):
            if pyautogui.locateOnScreen(f'screen/map/{name_map}_1.png',confidence=0.85-(i*0.01)):
                name_map+= '_1'
                break
            elif pyautogui.locateOnScreen(f'screen/map/{name_map}_2.png',confidence=0.85-(i*0.01)):
                name_map+= '_2'
                break
            elif i==9:
                print("erreur début de map")
                name_map=fct.error_start_map(self.map.list_map_T10)
                if name_map==False:
                    fct.leave_map()
                    return False
                print(f"map récupére {name_map}")
                break
            time.sleep(1)
        return name_map
        
        
    def run_map(self,wave,retry=True):
        WAIT_GOLD=50
        end=False
        name_map = self.find_map_on_start(self.name_map)
        if name_map == False: return False
        
        fct.prepar_run(self.num_perso)
        #run
        timer_gold=fct.timer.Timer(WAIT_GOLD)
        for i,point in enumerate(wave[name_map]):
            if pd.isna(point) :
                break
            x,y=map(int,re.findall(r'\d+',point))
            if i%5==0 and self.num_perso>=2:
                fct.keyboard('a',0.9)
            
            if not fct.clik_move_map(x,y):
                fct.clik_move_map(x,y)
                
            #pause entre deux points
            timer_move = fct.timer.Timer( fct.time_run(x,y)*1.1 if self.num_perso==1 else fct.time_run(x,y) )
            while timer_move.is_running():
                if keyboard.is_pressed('space'):
                    print(i,end=" ")
                    time.sleep(0.9)
                    while not keyboard.is_pressed('+'):
                        time.sleep(0.05)
                    fct.keyboard('a',0.2)
                    print("end_pause")
                    pyautogui.doubleClick(1200,1400)
                    timer_gold.relaunch()
                    if self.num_perso ==2:
                        fct.keyboard('a',0.7)
                                 
                if pyautogui.locateOnScreen(f'screen/map/ennemi.png',region=[1850,920,1900,1210],confidence=0.7,grayscale=True):
                    timer_gold.relaunch()
                if timer_gold.is_expire() and retry==True: 
                    end=True
                    break
                time.sleep(0.05)
            if end:break
            if fct.death():return False 
               
        if not fct.kill_boss(wave[name_map[:-2]+'_boss'],retry,self.num_perso):
            if retry:
                fct.escape()
                self.run_map(wave,False)

## Enregistrement

In [ ]:
pyautogui.leftClick(1200,1400)
time.sleep(0.5)
pyautogui.leftClick(1200,1400)

while 1:
    time.sleep(2)
    x,y=pyautogui.position()
    print(f'{x},{y}')        
     
    if 1:
        fct.keyboard('a',0.7)
       
    if not fct.clik_move_map(x,y): 
        fct.clik_move_map(x,y)

    


## Parcours un colonne

In [ ]:
cpt=0
pyautogui.leftClick(1200,1400)
time.sleep(0.5)
pyautogui.leftClick(1200,1400)

while 1:
    time.sleep(2.5)
    wave=pd.read_csv('wave.csv')
    n=wave.loc[cpt,"Lighting_1"]
    if pd.isna(n) == False:
        n=re.findall(r'\d+',n)
        if cpt%5==0:
            time.sleep(0.2)
            fct.keyboard('q',0.5)
            time.sleep(1)
            
           
        if not fct.clik_move_map(int(n[0]),int(n[1])):
            print(1)
            if not fct.clik_move_map(int(n[0]),int(n[1])):
                print(2)
                
                fct.clik_move_map(int(n[0]),int(n[1]))
       
        print(cpt,end=' ')
        cpt+=1

In [ ]:
import fct
reload(fct)
time.sleep(2)
m='Plague'

perso=Perso(1,raid=False)
perso.run_map(m,wave)


In [ ]:
#Team Viewer
pyautogui.leftClick(fct.resolution_scale("2520,1120"))
time.sleep(5)
pyautogui.leftClick(1536,792)

# Main

In [77]:
reload(fct)
list_map={'T10':["Lighting","Air","Brazier","Wanderer","Plague","Cycle","Flash"],#,"Greed","Shepherd"
          'T11':["Casthor","Boreal","Alyssa","Hamal","Capri","Miraseti","Acuben","Aquilla","Vesper","Spica","Leo"] }
fct.launch_remote_desk()   
perso=Perso(num_perso=3, list_map=list_map, can_run_T11=False)
alchimie=fct.alchimie.Alchimie(wave,perso.num_perso,T9_up=True,blue_charm=True,rare_charm=False)
    
while 1:
    if not perso.is_raid_time([15.35,19.15],[7.45,11.10]) and fct.launch_tower(wave,perso.num_perso):
        if perso.launch_map(): 
            perso.run_map(wave)
            fct.se_vider(sell=True)
            alchimie.run()
    else:
        fct.close_game()
        for i in range(60*5): time.sleep(1)
    

Looking For launch/reset.png
Looking For launch/party.png
Looking For launch/in_game.png
Close game for test_in_game
Looking For launch/reset.png
Looking For launch/party.png
Looking For launch/c_party.png
Looking For launch/c_party.png
Looking For launch/button.png
Looking For launch/Create.png
Looking For launch/ok.png
Looking For launch/move_to_party_area.png
Looking For launch/dungeon.png
Looking For launch/reset.png
Looking For launch/dungeon.png
Looking For launch/dungeon.png
Looking For launch/reset.png
Looking For launch/dungeon.png
Looking For launch/start_filtre.png
Looking For launch/select_all.png
Looking For map/ok_filtre.png
Looking For launch/croix.png
Looking For launch/reset.png
Looking For launch/waypoint.png
Looking For launch/tab_map.png
Looking For launch/boss.png
Looking For launch/tab_map.png
Looking For launch/reset.png
Looking For launch/storage.png
Looking For launch/reset.png
Looking For launch/ok.png
Looking For launch/reset.png
Looking For alchimie/atelier.

KeyboardInterrupt: 

In [47]:
perso.name_map

'Brazier'

## test

In [ ]:
reload(fct)
time.sleep(1)
#fct.close_game("no go tower",perso.num_perso)
alchimie=fct.alchimie.Alchimie(wave,perso.num_perso,T9_up=False,blue_charm=True,rare_charm=False)
perso.map.launch_map()

In [114]:
time.sleep(3)
perso=Perso(num_perso=3, list_map=list_map, region_map=True, can_run_T11=False)
perso.map.reset_region()

Looking For launch/start_filtre.png
Looking For launch/select_all.png
Looking For launch/select_all.png


In [ ]:
if pyautogui.locateOnScreen('screen/map/is_T11.png'):
    print("1")


In [ ]:
i=0
start_time = time.perf_counter()
while time.perf_counter()-start_time < fct.time_run(200,200):
    find = pyautogui.locateOnScreen(f'screen/map/ennemi.png', confidence=0.8,grayscale=True)
    if find:
        print(1,end='')
    else:
        print(" ",end="")
    i+=1
    if i ==150: 
        print(i)
        i=0

In [ ]:
if pyautogui.locateOnScreen(f'screen/launch/ok_map.png',confidence=0.7):print(1)

In [ ]:
print(pyautogui.screenshot().getpixel((i[0]+i[2]-5,i[1]+i[3]-5)))
print(pyautogui.screenshot().getpixel((2069+25,394+53)))
print(pyautogui.screenshot().getpixel((2171+25,394+53)))


In [ ]:
find = pyautogui.locateAllOnScreen(f'screen/map/Plague_b.png', confidence=0.99)
for i in find:
    print(i)
    print(pyautogui.screenshot().getpixel((i[0]+i[2]-5,i[1]+i[3]-5)))
    

## test dps

In [ ]:
# Initialisation de la variable qui contiendra la suite
suite = [1]

# Initialisation de la variable qui contiendra la somme
somme = 1

# Boucle pour générer les éléments de la suite
for i in range(1,11):
    # Calcul de l'élément suivant en décrémentant de 15%
    element = suite[i-1] * (0.9)
    # Ajout de l'élément à la suite
    suite.append(element)
    # Ajout de l'élément à la somme
    somme += element

# Affichage de la suite
print(suite)

# Affichage de la somme
print("La somme des éléments de la suite est :", somme*390)


In [ ]:
2676/2034

In [ ]:
(743*8+682*8)*0.25*1.25



## ACTE 10

In [ ]:
def run_acte_4_and_10(wave,retry=True,run_time=0):
    if retry:run_time=time.perf_counter()
    end=False
    fct.prepar_run(2)
    #run
    
    for i,n in enumerate(wave["Air_1"]):
        if pd.isna(n) :
            break
        x,y=re.findall(r'\d+',n)
        x=int(x)
        y=int(y)
        #perso2
        if i%5==0 :
            fct.keyboard('q',0.9)
        
        if not fct.clik_move_map(x,y):
            fct.clik_move_map(x,y)
            
        #pause entre deux points
        timer_move = fct.timer.Timer( fct.time_run(x,y)*1.1  )
        while timer_move.is_running():time.sleep(0.05)
        if fct.death():
            time.sleep(26)
            return False 
    
    fct.escape()
    time.sleep(4)
    pyautogui.leftClick(900,626,duration=0.3)
    fct.keyboard('f')
    pyautogui.leftClick(1150,626,duration=0.3)
    time.sleep(7)
    if fct.in_city():
        return True 
    
        
    time.sleep(4)
    fct.leave_map()
    time.sleep(2.5)
    return True

In [ ]:
#ACTE10
#perso=Perso(3,False)
reload(fct)
#cpt_no_go_map=0
while 1:
    fct.go_waypoint()
    fct.take_waypoint()
    pyautogui.leftClick(1053,1005,duration=0.3)
    pyautogui.leftClick(1900,900,duration=0.3)
    fct.click_icone('screen/launch/ok.png')
    fct.click_icone('screen/launch/reset.png')
    time.sleep(1.5)
    pyautogui.leftClick(1300,1400,duration=0.2)
    pyautogui.leftClick(1020,1200,duration=0.3)
    time.sleep(1.5)
    pyautogui.leftClick(1020,1200,duration=0.3)
    time.sleep(1.5)
    pyautogui.leftClick(750,727,duration=0.3)
    time.sleep(1.5)
    #Acte4
    pyautogui.leftClick(450,927,duration=0.3)
    #Acte10
    #pyautogui.leftClick(600,900,duration=0.3)
    fct.keyboard('f')
    time.sleep(5)
    #cpt_no_go_map+=1
    #if cpt_no_go_map==10: break
    
    if fct.in_map():
        #cpt_no_go_map=0
        
        run_acte_4_and_10(wave)
        #se vider
        if fct.in_city():
            if fct.go_blacksmith()==False:
                fct.go_blacksmith()
        time.sleep(0.5)
        if fct.go_waypoint() :
            fct.click_icone('screen/launch/icone_storage_acte10.png')
            time.sleep(3)
            fct.click_icone('screen/launch/storage.png',1)
            fct.click_icone('screen/launch/storage_clear.png',1)
            fct.click_icone('screen/launch/croix.png')
            time.sleep(0.5)    
        for i in range(np.random.randint(30, 120)):time.sleep(1)
        
        

    

## ACTE 11

In [ ]:
def run_acte_11(wave):
    fct.prepar_run(1)    
    for i,point in enumerate(wave["Shepherd_1"]):
        if pd.isna(point) :
            break
        x,y=map(int,re.findall(r'\d+',point))
        #perso2
        """ if i%5==0 :
            fct.keyboard('a',0.9)"""
        
        if not fct.clik_move_map(x,y):
            fct.clik_move_map(x,y)
            
        #pause entre deux points
        timer_move = fct.timer.Timer( fct.time_run(x,y)*1  )
        while timer_move.is_running():time.sleep(0.05)
        if fct.death():
            time.sleep(26)
            return False     
        
    time.sleep(4)
    fct.leave_map()
    time.sleep(2.5)
    return True

In [ ]:
reload(fct)
time.sleep(2)
while 1:
    timer_by_run = fct.timer.Timer( 60*7.3  )
    fct.go_waypoint()
    fct.take_waypoint()
    pyautogui.leftClick(1528,692,duration=0.3)
    #fct.click_icone('screen/Acte/Acte11_map.png')
    pyautogui.leftClick(1010,900,duration=0.3)
    fct.click_icone('screen/launch/ok.png')
    fct.click_icone('screen/Acte/move.png')
    
    time.sleep(10)
    fct.escape()
    
    fct.keyboard('f')
    time.sleep(5)
    #cpt_no_go_map+=1
    #if cpt_no_go_map==10: break
    
    if fct.in_map():
        #cpt_no_go_map=0
        
        run_acte_11(wave)
        #se vider
        if fct.in_city():
            fct.se_vider(sell=True)
        time.sleep(0.5)
        if fct.go_waypoint() :
            fct.click_icone('screen/launch/icone_storage_acte10.png')
            time.sleep(3)
            fct.click_icone('screen/launch/storage.png',1)
            fct.click_icone('screen/launch/storage_clear.png',1)
            fct.click_icone('screen/launch/croix.png')
            time.sleep(0.5)    
        while timer_by_run.is_running():time.sleep(1)
        
        